In [99]:
import sys
!{sys.executable} -m pip install requests websocket-client nbformat

In [100]:
import requests

# No auth since you passed token=''
url = 'http://localhost:8888'

# Start a kernel
resp = requests.post(f'{url}/api/kernels')
kernel_id = resp.json()['id']
print("Kernel ID:", kernel_id)

Kernel ID: 94ad266b-38c0-4714-9a30-67bf91b43507


In [101]:
import websocket
import uuid
import json

def send_execute_request(ws, code, kernel_id):
    msg_id = str(uuid.uuid4())
    msg = {
        "header": {
            "msg_id": msg_id,
            "username": "username",
            "session": str(uuid.uuid4()),
            "msg_type": "execute_request",
            "version": "5.3",
        },
        "parent_header": {},
        "metadata": {},
        "content": {
            "code": code,
            "silent": False,
        },
        "channel": "shell"
    }
    ws.send(json.dumps(msg))
    return msg_id

In [102]:
import nbformat

def append_code_cell_with_output_remote(jupyter_url, notebook_name, code, output):
    # 1. Read the notebook from the running Jupyter server
    response = requests.get(f"{jupyter_url}/api/contents/{notebook_name}")
    response.raise_for_status()
    content = response.json()["content"]
    
    # 2. Load notebook into nbformat object
    nb = nbformat.from_dict(content)

    # 3. Append new code cell and include kernel output
    new_cell = nbformat.v4.new_code_cell(source=code)
    # Build output object properly depending on type
    if output:
        output_type = output.get("output_type")
        if output_type == "execute_result":
            new_cell.outputs = [
                nbformat.v4.new_output(
                    output_type="execute_result",
                    data=output["data"],
                    metadata=output.get("metadata", {}),
                    execution_count=output.get("execution_count")
                )
            ]
        elif output_type == "stream":
            new_cell.outputs = [
                nbformat.v4.new_output(
                    output_type="stream",
                    name=output.get("name", "stdout"),
                    text=output.get("text", "")
                )
            ]
        elif output_type == "display_data":
            new_cell.outputs = [
                nbformat.v4.new_output(
                    output_type="display_data",
                    data=output.get("data", {}),
                    metadata=output.get("metadata", {})
                )
            ]
    new_cell.execution_count = len([c for c in nb.cells if c.cell_type == "code"]) + 1    
    nb.cells.append(new_cell)
    
    # 4. Push the modified notebook back
    # Strip unsupported fields for older Jupyter server
    for cell in nb.cells:
        if "id" in cell:
            del cell["id"]
    response = requests.put(
        f"{jupyter_url}/api/contents/{notebook_name}",
        json={
            "type": "notebook",
            "format": "json",
            "content": nb
        }
    )
    response.raise_for_status()


In [103]:
notebook_path = "test.ipynb"
code_to_run = "12 + 10"

# Step 1: Connect to WebSocket
ws_url = f"ws://localhost:8888/api/kernels/{kernel_id}/channels"
ws = websocket.create_connection(ws_url)

# Step 2: Send execute request
msg_id = send_execute_request(ws, code_to_run, kernel_id)

# Step 3: Wait for output
while True:
    msg = json.loads(ws.recv())
    if msg.get("parent_header", {}).get("msg_id") == msg_id:
        if msg["msg_type"] in ["execute_result", "stream", "display_data"]:
            execution_output = msg["content"]
            execution_output["output_type"] = msg["msg_type"]
        elif msg["msg_type"] == "status" and msg["content"]["execution_state"] == "idle":
            break
        
# 4. Append code & output to container’s notebook via API
append_code_cell_with_output_remote("http://localhost:8888", notebook_path, code_to_run, execution_output)

print(execution_output)

{'data': {'text/plain': '22'}, 'metadata': {}, 'execution_count': 1, 'output_type': 'execute_result'}
